In [4]:
import anyio
from anyio.streams.memory import MemoryObjectSendStream

import openai
from utils.secret_key import OPENAI_KEY

client = openai.AsyncOpenAI(api_key=OPENAI_KEY)

async def async_gpt(prompt, stream: MemoryObjectSendStream[str]):
    print("Calling GPT")
    messages = [{"role": "user", "content": prompt}]
    response = await client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
    )
    # Instead of returning we send the result to the send stream
    await stream.send(response.choices[0].message.content)

async def receiver(receive_stream):
    print("Starting receiving")
    async with receive_stream:
        async for item in receive_stream:
            print('received:', item)
    print('Ending Recieiving')


prompts=["Create a haiku", "What's 2 to the power of 40", "Summarize the history of Australia"]


async def main():
    # The [str] specifies the type of the objects being passed through the
    # memory object stream. This is a bit of trick, as create_memory_object_stream
    # is actually a class masquerading as a function.
    send_stream, receive_stream = anyio.create_memory_object_stream[str]()

    # Create the task group and pass the send stream
    async with anyio.create_task_group() as tg:
        tg.start_soon(receiver, receive_stream)

        print("send prompts")
        for prompt in prompts:
            tg.start_soon(async_gpt, prompt, send_stream)

    print("closing stream")
    await send_stream.aclose()


    print("finished")

# dont need to anyio.run in jupyter
await main()

send prompts
Starting receiving
Calling GPT
Calling GPT
Calling GPT
received: Whispers of the breeze,  
Autumn leaves dance to the ground,  
Silent Earth asleep.
received: 2 to the power of 40 is 1,099,511,627,776.
received: Australia's history is a rich tapestry that spans tens of thousands of years, marked by the diverse cultures of its Indigenous peoples, European exploration, colonization, and the development of a modern nation-state.

**Indigenous Australia:** 
- Indigenous Australians, including Aboriginal and Torres Strait Islander peoples, are believed to have arrived in Australia over 65,000 years ago. 
- They developed complex societies with deep spiritual connections to the land, rich traditions, and diverse languages and cultures.

**European Exploration and Colonization:**
- The first recorded European contact occurred in 1606 when Dutch explorer Willem Janszoon reached the Cape York Peninsula.
- In 1770, British explorer Captain James Cook charted the east coast of Austra